In [ ]:
# ============================================================
# CICIDS — COMPLETE TRAINING PIPELINE (Safe Encoders Version)
# ============================================================

import os
import time
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import pickle

# ============================================================
# CONFIG
# ============================================================

TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

MODEL_DIR = "model_cicids"
OUT_DIR = "CICIDS_output"

os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

LABEL_COL = "Attack"

CUCKOO_PARTICLES = 25
CUCKOO_ITER = 1
RANDOM_STATE = 42
SMOTE_K_NEIGH = 3


# ============================================================
# LOAD DATA
# ============================================================

train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

print("Train:", train_df.shape, "Test:", test_df.shape)

# DROP NON-FEATURE COLUMNS (Flow ID, IPs, timestamps)
drop_keywords = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]

drop_cols = [c for c in train_df.columns
             if any(k in c.lower() for k in drop_keywords)]

train_df.drop(columns=drop_cols, inplace=True, errors="ignore")
test_df.drop(columns=drop_cols, inplace=True, errors="ignore")

print("After dropping non-features:")
print("Train:", train_df.shape)
print("Test :", test_df.shape)


# ============================================================
# TARGET ENCODING
# ============================================================

all_labels = pd.concat([train_df[LABEL_COL], test_df[LABEL_COL]], axis=0).astype(str)
classes = sorted(all_labels.unique())
target_map = {c: i for i, c in enumerate(classes)}

train_df[LABEL_COL] = train_df[LABEL_COL].astype(str).map(target_map)
test_df[LABEL_COL]  = test_df[LABEL_COL].astype(str).map(target_map)

# Save target encoder
pickle.dump(target_map, open(f"{MODEL_DIR}/target_encoder.pkl", "wb"))


# ============================================================
# SAFE CATEGORICAL ENCODER
# ============================================================

def safe_label_encode(series):
    uniq = sorted(series.astype(str).unique())
    mapping = {v: i for i, v in enumerate(uniq)}

    def transform(vals):
        return np.array([mapping.get(str(v), len(mapping)) for v in vals], dtype=int)

    return mapping, transform


encoders = {}
cat_cols = [c for c in train_df.columns if train_df[c].dtype == object and c != LABEL_COL]

# Fit on training → safe encode testing
for c in cat_cols:
    mapping, transform = safe_label_encode(train_df[c])

    train_df[c] = transform(train_df[c])
    test_df[c]  = transform(test_df[c])   # unseen values handled safely

    encoders[c] = {"mapping": mapping}


pickle.dump(encoders, open(f"{MODEL_DIR}/encoders.pkl", "wb"))


# ============================================================
# SPLIT + SCALE
# ============================================================

X_train_df = train_df.drop(columns=[LABEL_COL])
y_train = train_df[LABEL_COL]

X_test_df = test_df.drop(columns=[LABEL_COL])
y_test = test_df[LABEL_COL]

X_train_df, X_test_df = X_train_df.align(X_test_df, join="left", axis=1, fill_value=0)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_df)
X_test  = scaler.transform(X_test_df)

feat_names = np.array(X_train_df.columns)

pickle.dump(scaler, open(f"{MODEL_DIR}/scaler.pkl", "wb"))


# ============================================================
# CUCKOO SEARCH FEATURE SELECTION
# ============================================================

def cuckoo_search(X, y, particles=25, iterations=1):
    rng = np.random.RandomState(RANDOM_STATE)
    n_features = X.shape[1]

    def fitness(mask):
        idx = np.where(mask)[0]
        if len(idx) == 0:
            return 0
        clf = RandomForestClassifier(n_estimators=40, max_depth=12, random_state=RANDOM_STATE)
        skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)
        scores = []
        for tr, va in skf.split(X, y):
            clf.fit(X[tr][:, idx], y[tr])
            scores.append(clf.score(X[va][:, idx], y[va]))
        return np.mean(scores)

    nests = rng.rand(particles, n_features) > 0.5
    fitnesses = np.array([fitness(n) for n in nests])

    best_idx = np.argmax(fitnesses)
    best_mask = nests[best_idx].copy()
    best_fit = fitnesses[best_idx]

    beta = 1.5
    sigma = (math.gamma(1+beta) * math.sin(math.pi*beta/2)
             / (math.gamma((1+beta)/2) * beta * 2**((beta-1)/2))) ** (1/beta)

    def levy():
        u = rng.normal(0, sigma, size=n_features)
        v = rng.normal(0, 1, size=n_features)
        return u / (np.abs(v) ** (1/beta))

    print("Initial best fitness:", best_fit)

    for it in range(iterations):
        for i in range(particles):
            step = levy()
            step = (step - step.min()) / (step.max() - step.min() + 1e-9)
            new_mask = nests[i] ^ (step > 0.5)
            new_fit = fitness(new_mask)

            if new_fit > fitnesses[i]:
                nests[i] = new_mask
                fitnesses[i] = new_fit
                if new_fit > best_fit:
                    best_fit = new_fit
                    best_mask = new_mask.copy()

        print(f"Iteration {it+1}/{iterations}: best fitness = {best_fit:.4f}")

    return feat_names[best_mask].tolist()


selected_features = cuckoo_search(X_train, y_train)
pickle.dump(selected_features, open(f"{MODEL_DIR}/feature_list.pkl", "wb"))

print("\nSelected Features:", selected_features)


# ============================================================
# TRAIN FINAL MODEL (XGBOOST)
# ============================================================

sel_idx = [np.where(feat_names == f)[0][0] for f in selected_features]
Xtr_sel = X_train[:, sel_idx]
Xte_sel = X_test[:, sel_idx]

sm = SMOTE(k_neighbors=SMOTE_K_NEIGH, random_state=RANDOM_STATE)
Xtr_sm, ytr_sm = sm.fit_resample(Xtr_sel, y_train)

xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric="mlogloss",
    tree_method="hist",
    n_estimators=150,
    max_depth=12,
    random_state=RANDOM_STATE
)
xgb.fit(Xtr_sm, ytr_sm)

pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl", "wb"))


# ============================================================
# SAVE LIME BACKGROUND
# ============================================================

bg_idx = np.random.choice(len(X_train), size=min(1200, len(X_train)), replace=False)
lime_bg = X_train[bg_idx]

pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl", "wb"))

print("LIME background saved!")


# ============================================================
# EVALUATE MODEL
# ============================================================

y_pred = xgb.predict(Xte_sel)

acc = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted")

print("\n===== CICIDS MODEL RESULTS =====")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 Score :", f1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=list(target_map.keys())))


Train: (1469979, 106) Test: (629992, 106)
After dropping non-features:
Train: (1469979, 102)
Test : (629992, 102)
Initial best fitness: 0.9997428534693352
Iteration 1/1: best fitness = 0.9997

Selected Features: ['Src Port', 'Dst Port', 'Protocol', 'Flow Duration', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Min', 'Fwd Packet Length Std', 'Bwd Packet Length Mean', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Std', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Bwd Header Length', 'Fwd Packets/s', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'SYN Flag Count', 'RST Flag Count', 'ACK Flag Count', 'CWR Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg', 'Bwd Packet/Bulk Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'FWD Init Win Bytes', 'Bwd Init

In [1]:
# ============================================================
# NEXT STEP: BUILD FINAL CICIDS MODEL PACKAGE FOR FLASK
# ============================================================

import pickle
import numpy as np
import pandas as pd
import os

print("Preparing CICIDS final model package...\n")

# Ensure model directory exists
os.makedirs(MODEL_DIR, exist_ok=True)

# ------------------------------------------------------------
# 1) Validate Selected Features
# ------------------------------------------------------------
print("[1] Validating selected features...")
selected_features = pickle.load(open(f"{MODEL_DIR}/feature_list.pkl", "rb"))
print("Selected:", len(selected_features), "features")
print(selected_features[:20], "...")

# Build selected index list
sel_idx = [np.where(feat_names == f)[0][0] for f in selected_features if f in feat_names]

# Safety check
if len(sel_idx) == 0:
    raise ValueError("ERROR: No selected features matched training columns! Check feature names.")

print("Selected indices:", sel_idx[:20], "\n")


# ------------------------------------------------------------
# 2) Save final scaler, encoders, feature list (already saved earlier, but re-check)
# ------------------------------------------------------------
print("[2] Re-checking preprocessing objects...")

print("Scaler exists:", os.path.exists(f"{MODEL_DIR}/scaler.pkl"))
print("Encoders exist:", os.path.exists(f"{MODEL_DIR}/encoders.pkl"))
print("Target encoder exists:", os.path.exists(f"{MODEL_DIR}/target_encoder.pkl"))

# Reload safely
scaler = pickle.load(open(f"{MODEL_DIR}/scaler.pkl", "rb"))
encoders = pickle.load(open(f"{MODEL_DIR}/encoders.pkl", "rb"))
target_map = pickle.load(open(f"{MODEL_DIR}/target_encoder.pkl", "rb"))

print("Loaded target classes:", target_map, "\n")


# ------------------------------------------------------------
# 3) Create REVERSE target mapping (for Flask predictions)
# ------------------------------------------------------------
inv_target_map = {v: k for k, v in target_map.items()}
pickle.dump(inv_target_map, open(f"{MODEL_DIR}/target_inverse.pkl", "wb"))
print("[3] Saved target_inverse.pkl for label decoding.\n")


# ------------------------------------------------------------
# 4) Save final XGBoost model
# ------------------------------------------------------------
print("[4] Saving XGBoost model...")
pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl", "wb"))
print("Saved xgb_model.pkl\n")


# ------------------------------------------------------------
# 5) Save LIME Background dataset
# ------------------------------------------------------------
print("[5] Saving LIME background...")
pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl", "wb"))
print("Saved lime_background.pkl\n")


# ------------------------------------------------------------
# 6) Save final feature list (correct order)
# ------------------------------------------------------------
print("[6] Saving final ordered feature_list.pkl...")
pickle.dump(selected_features, open(f"{MODEL_DIR}/feature_list.pkl", "wb"))
print("Saved feature_list.pkl\n")


# ------------------------------------------------------------
# 7) Build CICIDS pipeline summary
# ------------------------------------------------------------
summary = {
    "num_features": len(selected_features),
    "features": selected_features,
    "target_classes": target_map,
    "selected_indexes": sel_idx
}

pickle.dump(summary, open(f"{MODEL_DIR}/summary.pkl", "wb"))
print("[7] Saved summary.pkl\n")

print("===================================================")
print(" ✔✔✔ CICIDS WEB PACKAGE READY FOR FLASK ✔✔✔ ")
print("===================================================")
print(f"Files saved inside: {MODEL_DIR}")
print(os.listdir(MODEL_DIR))
print("===================================================")


Preparing CICIDS final model package...



NameError: name 'MODEL_DIR' is not defined

In [2]:
# ============================================================
# CICIDS — FINAL PACKAGING CELL (Self-contained)
# ============================================================

import os
import pickle
import numpy as np
import pandas as pd

MODEL_DIR = "model_cicids"
TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

print("Preparing CICIDS final model package...\n")

# ------------------------------------------------------------
# 1) Reload necessary training objects from disk
# ------------------------------------------------------------

if not os.path.exists(MODEL_DIR):
    raise RuntimeError(f"ERROR: {MODEL_DIR} does not exist. Run training first.")

# Load all saved objects
try:
    scaler = pickle.load(open(f"{MODEL_DIR}/scaler.pkl", "rb"))
    encoders = pickle.load(open(f"{MODEL_DIR}/encoders.pkl", "rb"))
    target_map = pickle.load(open(f"{MODEL_DIR}/target_encoder.pkl", "rb"))
    selected_features = pickle.load(open(f"{MODEL_DIR}/feature_list.pkl", "rb"))
except Exception as e:
    raise RuntimeError("Some required model files are missing.\n" + str(e))

print("Loaded scaler, encoders, target encoder, and feature list.\n")

# For index mapping, load training columns from actual dataset
train_df = pd.read_csv(TRAIN_PATH)

# Remove irrelevant columns again to match training
drop_keywords = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]
drop_cols = [c for c in train_df.columns if any(k in c.lower() for k in drop_keywords)]
train_df.drop(columns=drop_cols, inplace=True, errors="ignore")

# Extract feature names
feat_names = np.array(train_df.drop(columns=["Attack"]).columns)

print("Training feature columns loaded:", len(feat_names))


# ------------------------------------------------------------
# 2) Convert selected feature names → column indexes
# ------------------------------------------------------------

sel_idx = []
for f in selected_features:
    if f in feat_names:
        sel_idx.append(np.where(feat_names == f)[0][0])
    else:
        print("[WARNING] Feature missing in training columns:", f)

if len(sel_idx) == 0:
    raise RuntimeError("ERROR: No selected features matched training columns!")

print("Selected indexes:", sel_idx[:20], "...\n")


# ------------------------------------------------------------
# 3) Load trained model OR tell user to train if missing
# ------------------------------------------------------------

model_path = f"{MODEL_DIR}/xgb_model.pkl"

if not os.path.exists(model_path):
    raise RuntimeError("ERROR: xgb_model.pkl not found. You must finish training cell first.")

xgb = pickle.load(open(model_path, "rb"))
print("Loaded XGBoost model.\n")


# ------------------------------------------------------------
# 4) Build reverse target mapping for Flask
# ------------------------------------------------------------

inv_target_map = {v: k for k, v in target_map.items()}
pickle.dump(inv_target_map, open(f"{MODEL_DIR}/target_inverse.pkl", "wb"))

print("Saved target_inverse.pkl\n")


# ------------------------------------------------------------
# 5) Save LIME background (reconstruct safely)
# ------------------------------------------------------------

# Load full training data again for background sampling
train_df = pd.read_csv(TRAIN_PATH)
train_df.drop(columns=drop_cols, inplace=True, errors="ignore")

X_train = train_df.drop(columns=["Attack"]).values

# Limit background to prevent RAM crash
bg_size = min(1200, len(X_train))
bg_idx = np.random.choice(len(X_train), size=bg_size, replace=False)

lime_background = X_train[bg_idx]
pickle.dump(lime_background, open(f"{MODEL_DIR}/lime_background.pkl", "wb"))

print(f"LIME background saved → {bg_size} samples\n")


# ------------------------------------------------------------
# 6) Save summary
# ------------------------------------------------------------

summary = {
    "num_features": len(selected_features),
    "features": selected_features,
    "target_classes": target_map,
    "selected_indexes": sel_idx
}

pickle.dump(summary, open(f"{MODEL_DIR}/summary.pkl", "wb"))

print("Saved summary.pkl\n")


print("===================================================")
print("   ✔✔✔ CICIDS WEB PACKAGE READY FOR FLASK ✔✔✔")
print("===================================================")
print("Files in model_cicids directory:")
print(os.listdir(MODEL_DIR))
print("===================================================")


Preparing CICIDS final model package...

Loaded scaler, encoders, target encoder, and feature list.

Training feature columns loaded: 101
Selected indexes: [0, 1, 2, 3, 6, 7, 9, 11, 14, 16, 17, 18, 22, 23, 24, 25, 26, 27, 29, 31] ...



RuntimeError: ERROR: xgb_model.pkl not found. You must finish training cell first.

In [3]:
# ============================================================
# FAST CICIDS TRAINING PIPELINE — Produces model in minutes
# ============================================================

import os, math, warnings, pickle, random
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

MODEL_DIR = "model_cicids"
os.makedirs(MODEL_DIR, exist_ok=True)

TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

print("Loading CSVs...")
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

# ------------------------------------------------------------
# Drop heavy unused columns
# ------------------------------------------------------------
drop_keys = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]
drop_cols = [c for c in train_df.columns if any(k in c.lower() for k in drop_keys)]
train_df.drop(columns=drop_cols, inplace=True, errors="ignore")
test_df.drop(columns=drop_cols, inplace=True, errors="ignore")

LABEL_COL = "Attack"

# ------------------------------------------------------------
# Encode output label
# ------------------------------------------------------------
all_labels = pd.concat([train_df[LABEL_COL], test_df[LABEL_COL]], axis=0).astype(str)
classes = sorted(all_labels.unique())
target_map = {c:i for i,c in enumerate(classes)}

train_df[LABEL_COL] = train_df[LABEL_COL].astype(str).map(target_map)
test_df[LABEL_COL]  = test_df[LABEL_COL].astype(str).map(target_map)

pickle.dump(target_map, open(f"{MODEL_DIR}/target_encoder.pkl","wb"))

# ------------------------------------------------------------
# Encode categorical features (safe)
# ------------------------------------------------------------
def safe_encode(series):
    uniq = sorted(series.astype(str).unique())
    mapping = {v:i for i,v in enumerate(uniq)}
    def transform(vals):
        return np.array([mapping.get(str(v), len(mapping)) for v in vals])
    return mapping, transform

encoders = {}
cat_cols = [c for c in train_df.columns if train_df[c].dtype==object and c!=LABEL_COL]

for c in cat_cols:
    mapping, transform = safe_encode(train_df[c])
    train_df[c] = transform(train_df[c])
    test_df[c]  = transform(test_df[c])
    encoders[c] = {"mapping": mapping}

pickle.dump(encoders, open(f"{MODEL_DIR}/encoders.pkl","wb"))

# ------------------------------------------------------------
# Build X & y
# ------------------------------------------------------------
X_train_full = train_df.drop(columns=[LABEL_COL])
y_train_full = train_df[LABEL_COL]

X_test = test_df.drop(columns=[LABEL_COL])
y_test = test_df[LABEL_COL]

# ------------------------------------------------------------
# SPEED FIX — Use 150k rows only for training
# ------------------------------------------------------------
train_sample = min(150000, len(X_train_full))
idx = np.random.choice(len(X_train_full), size=train_sample, replace=False)
X_train = X_train_full.iloc[idx]
y_train = y_train_full.iloc[idx]

print(f"Training on reduced dataset: {X_train.shape}")

# ------------------------------------------------------------
# Scale
# ------------------------------------------------------------
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

pickle.dump(scaler, open(f"{MODEL_DIR}/scaler.pkl","wb"))

# ------------------------------------------------------------
# SMOTE (small k to avoid explosion)
# ------------------------------------------------------------
sm = SMOTE(k_neighbors=2, random_state=42)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print("After SMOTE:", X_sm.shape)

# ------------------------------------------------------------
# FAST XGBOOST MODEL
# ------------------------------------------------------------
xgb = XGBClassifier(
    tree_method="hist",
    n_estimators=80,
    max_depth=10,
    learning_rate=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
    eval_metric="mlogloss",
    n_jobs=-1,
    random_state=42
)

xgb.fit(X_sm, y_sm)
pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl","wb"))

print("\nModel saved → model_cicids/xgb_model.pkl")

# ------------------------------------------------------------
# Save Feature List
# ------------------------------------------------------------
feat_list = list(X_train_full.columns)
pickle.dump(feat_list, open(f"{MODEL_DIR}/feature_list.pkl","wb"))

# ------------------------------------------------------------
# Save LIME background (small)
# ------------------------------------------------------------
bg_idx = np.random.choice(len(X_train), size=800, replace=False)
lime_bg = X_train[bg_idx]
pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl","wb"))

print("LIME background saved.")

# ------------------------------------------------------------
# Evaluate quickly
# ------------------------------------------------------------
pred = xgb.predict(X_test)
acc = accuracy_score(y_test, pred)

print("\nFinal CICIDS Model Accuracy:", acc)
print("✔ Training complete.")


Loading CSVs...
Training on reduced dataset: (150000, 101)


ValueError: Expected n_neighbors <= n_samples_fit, but n_neighbors = 3, n_samples_fit = 1, n_samples = 1

In [4]:
# ============================================================
# FAST + SAFE CICIDS TRAINING PIPELINE (SMOTE error-proof)
# ============================================================

import os, warnings, pickle
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

MODEL_DIR = "model_cicids"
os.makedirs(MODEL_DIR, exist_ok=True)

TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

print("\nLoading CICIDS CSV files...")
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

LABEL_COL = "Attack"

# ------------------------------------------------------------
# 1. DROP NON-FEATURE COLUMNS
# ------------------------------------------------------------
drop_keys = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]
drop_cols = [c for c in train_df.columns if any(k in c.lower() for k in drop_keys)]

train_df.drop(columns=drop_cols, inplace=True, errors="ignore")
test_df.drop(columns=drop_cols, inplace=True, errors="ignore")

print("After drop →", train_df.shape)

# ------------------------------------------------------------
# 2. TARGET ENCODING
# ------------------------------------------------------------
all_labels = pd.concat([train_df[LABEL_COL], test_df[LABEL_COL]]).astype(str)
classes = sorted(all_labels.unique())
target_map = {c:i for i,c in enumerate(classes)}

train_df[LABEL_COL] = train_df[LABEL_COL].astype(str).map(target_map)
test_df[LABEL_COL]  = test_df[LABEL_COL].astype(str).map(target_map)

pickle.dump(target_map, open(f"{MODEL_DIR}/target_encoder.pkl","wb"))

# ------------------------------------------------------------
# 3. SAFE ENCODING OF CATEGORICAL COLUMNS
# ------------------------------------------------------------
def safe_encode(series):
    uniq = sorted(series.astype(str).unique())
    mapping = {v:i for i,v in enumerate(uniq)}
    def transform(values):
        return np.array([mapping.get(str(v), len(mapping)) for v in values])
    return mapping, transform

encoders = {}
cat_cols = [c for c in train_df.columns if train_df[c].dtype == object and c != LABEL_COL]

for c in cat_cols:
    mapping, transform = safe_encode(train_df[c])
    train_df[c] = transform(train_df[c])
    test_df[c]  = transform(test_df[c])
    encoders[c] = {"mapping": mapping}

pickle.dump(encoders, open(f"{MODEL_DIR}/encoders.pkl","wb"))

# ------------------------------------------------------------
# 4. SPLIT X/y
# ------------------------------------------------------------
X_train_full = train_df.drop(columns=[LABEL_COL])
y_train_full = train_df[LABEL_COL]

X_test = test_df.drop(columns=[LABEL_COL])
y_test = test_df[LABEL_COL]

# ------------------------------------------------------------
# 5. REDUCE DATASET FOR SPEED
# ------------------------------------------------------------
N = min(150000, len(X_train_full))
idx = np.random.choice(len(X_train_full), size=N, replace=False)

X_train = X_train_full.iloc[idx].copy()
y_train = y_train_full.iloc[idx].copy()

print(f"Training reduced dataset → {X_train.shape}")

# ------------------------------------------------------------
# 6. REMOVE RARE CLASSES (< 3 samples) → Fixes SMOTE error
# ------------------------------------------------------------
vc = y_train.value_counts()
rare = vc[vc < 3].index.tolist()

if rare:
    print("Dropping rare classes (<3 samples):", rare)
    keep_idx = ~y_train.isin(rare)
    X_train = X_train[keep_idx]
    y_train = y_train[keep_idx]

print("After rare drop →", X_train.shape)

# ------------------------------------------------------------
# 7. SCALE
# ------------------------------------------------------------
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
pickle.dump(scaler, open(f"{MODEL_DIR}/scaler.pkl","wb"))

# ------------------------------------------------------------
# 8. SAFE SMOTE (k_neighbors=1)
# ------------------------------------------------------------
print("\nApplying SMOTE safely...")

sm = SMOTE(k_neighbors=1, random_state=42)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print("After SMOTE →", X_sm.shape)

# ------------------------------------------------------------
# 9. TRAIN XGBOOST (FAST)
# ------------------------------------------------------------
xgb = XGBClassifier(
    tree_method="hist",
    n_estimators=80,
    max_depth=10,
    learning_rate=0.08,
    colsample_bytree=0.7,
    subsample=0.7,
    eval_metric="mlogloss",
    random_state=42,
    n_jobs=-1,
)

xgb.fit(X_sm, y_sm)
pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl","wb"))
print("\n✔ Model saved: model_cicids/xgb_model.pkl")

# ------------------------------------------------------------
# 10. SAVE FEATURE LIST
# ------------------------------------------------------------
feat_list = list(X_train_full.columns)
pickle.dump(feat_list, open(f"{MODEL_DIR}/feature_list.pkl","wb"))

# ------------------------------------------------------------
# 11. SAVE LIME BACKGROUND
# ------------------------------------------------------------
bg_idx = np.random.choice(len(X_train), size=min(1000, len(X_train)), replace=False)
lime_bg = X_train[bg_idx]
pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl","wb"))

print("✔ LIME background saved.")

# ------------------------------------------------------------
# 12. Evaluation
# ------------------------------------------------------------
pred = xgb.predict(X_test)
acc = accuracy_score(y_test, pred)

print("\n========== FINAL CICIDS ACCURACY ==========")
print("Accuracy:", acc)
print("===========================================")



Loading CICIDS CSV files...
After drop → (1469979, 102)
Training reduced dataset → (150000, 101)
Dropping rare classes (<3 samples): [25, 13, 14, 23, 20, 24, 15]
After rare drop → (149992, 101)

Applying SMOTE safely...
After SMOTE → (2262660, 101)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19], got [ 0  1  2  3  4  5  6  7  8  9 10 11 12 16 17 18 19 21 22 26]

In [5]:
# ============================================================
# FINAL — ERROR-PROOF CICIDS TRAINING BLOCK
# ============================================================

import os, warnings, pickle
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

MODEL_DIR = "model_cicids"
os.makedirs(MODEL_DIR, exist_ok=True)

TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

LABEL_COL = "Attack"

print("\nLoading CICIDS...")
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)


# ------------------------------------------------------------
# 1. Drop non-feature columns
# ------------------------------------------------------------
drop_keys = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]
drop_cols = [c for c in train_df.columns if any(k in c.lower() for k in drop_keys)]

train_df.drop(columns=drop_cols, inplace=True, errors="ignore")
test_df.drop(columns=drop_cols, inplace=True, errors="ignore")


# ------------------------------------------------------------
# 2. Build label mapping from TRAIN ONLY (important!)
# ------------------------------------------------------------
train_labels = train_df[LABEL_COL].astype(str)
unique_train_labels = sorted(train_labels.unique())

target_map = {label: i for i, label in enumerate(unique_train_labels)}

train_df[LABEL_COL] = train_df[LABEL_COL].astype(str).map(target_map)

# Test might contain unseen labels → remove them safely
test_df = test_df[test_df[LABEL_COL].astype(str).isin(unique_train_labels)].copy()
test_df[LABEL_COL] = test_df[LABEL_COL].astype(str).map(target_map)

pickle.dump(target_map, open(f"{MODEL_DIR}/target_encoder.pkl","wb"))

print("Labels used:", target_map)


# ------------------------------------------------------------
# 3. SAFE categorical encoding
# ------------------------------------------------------------
def safe_encode(series):
    uniq = sorted(series.astype(str).unique())
    mapping = {v:i for i,v in enumerate(uniq)}
    def tf(values):
        return np.array([mapping.get(str(v), len(mapping)) for v in values])
    return mapping, tf

encoders = {}
cat_cols = [c for c in train_df.columns if train_df[c].dtype == object and c != LABEL_COL]

for c in cat_cols:
    mapping, tf = safe_encode(train_df[c])
    train_df[c] = tf(train_df[c])
    test_df[c]  = tf(test_df[c])
    encoders[c] = mapping

pickle.dump(encoders, open(f"{MODEL_DIR}/encoders.pkl","wb"))


# ------------------------------------------------------------
# 4. Reduce dataset size (150k rows)
# ------------------------------------------------------------
N = min(150000, len(train_df))
idx = np.random.choice(len(train_df), size=N, replace=False)

train_df = train_df.iloc[idx].copy()


# ------------------------------------------------------------
# 5. Remove rare classes (<3 samples) BEFORE training
# ------------------------------------------------------------
vc = train_df[LABEL_COL].value_counts()
rare = vc[vc < 3].index.tolist()

if rare:
    print("Dropping rare classes:", rare)
    train_df = train_df[~train_df[LABEL_COL].isin(rare)].copy()

# Re-check valid class list
valid_classes = sorted(train_df[LABEL_COL].unique())
print("Remaining classes:", valid_classes)


# ------------------------------------------------------------
# 6. Split into X/y and scale
# ------------------------------------------------------------
X_train = train_df.drop(columns=[LABEL_COL])
y_train = train_df[LABEL_COL]

X_test = test_df.drop(columns=[LABEL_COL])
y_test = test_df[LABEL_COL]

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

pickle.dump(scaler, open(f"{MODEL_DIR}/scaler.pkl","wb"))


# ------------------------------------------------------------
# 7. Safe SMOTE (k=1)
# ------------------------------------------------------------
print("\nApplying SMOTE...")
sm = SMOTE(k_neighbors=1, random_state=42)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print("After SMOTE:", X_sm.shape)


# ------------------------------------------------------------
# 8. Train XGBoost WITH corrected class list
# ------------------------------------------------------------
print("\nTraining XGBoost...")

xgb = XGBClassifier(
    tree_method="hist",
    n_estimators=120,
    max_depth=12,
    learning_rate=0.1,
    subsample=0.7,
    colsample_bytree=0.7,
    eval_metric="mlogloss",
    n_jobs=-1,
    random_state=42
)

# IMPORTANT: manually set expected classes
xgb.classes_ = np.array(valid_classes)

xgb.fit(X_sm, y_sm)

pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl","wb"))
print("✔ Model saved.")


# ------------------------------------------------------------
# 9. Save feature list
# ------------------------------------------------------------
feat_list = list(train_df.drop(columns=[LABEL_COL]).columns)
pickle.dump(feat_list, open(f"{MODEL_DIR}/feature_list.pkl","wb"))

print("✔ Feature list saved.")


# ------------------------------------------------------------
# 10. Save LIME background
# ------------------------------------------------------------
bg_idx = np.random.choice(len(X_train), size=min(800, len(X_train)), replace=False)
lime_bg = X_train[bg_idx]
pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl","wb"))

print("✔ LIME background saved.")


# ------------------------------------------------------------
# 11. Evaluate
# ------------------------------------------------------------
pred = xgb.predict(X_test)
acc = accuracy_score(y_test, pred)

print("\n========== FINAL ACCURACY ==========")
print("Accuracy:", acc)
print("=====================================")



Loading CICIDS...
Labels used: {'BENIGN': 0, 'Botnet': 1, 'Botnet - Attempted': 2, 'DDoS': 3, 'DoS GoldenEye': 4, 'DoS GoldenEye - Attempted': 5, 'DoS Hulk': 6, 'DoS Hulk - Attempted': 7, 'DoS Slowhttptest': 8, 'DoS Slowhttptest - Attempted': 9, 'DoS Slowloris': 10, 'DoS Slowloris - Attempted': 11, 'FTP-Patator': 12, 'FTP-Patator - Attempted': 13, 'Heartbleed': 14, 'Infiltration': 15, 'Infiltration - Attempted': 16, 'Infiltration - Portscan': 17, 'Portscan': 18, 'SSH-Patator': 19, 'SSH-Patator - Attempted': 20, 'Web Attack - Brute Force': 21, 'Web Attack - Brute Force - Attempted': 22, 'Web Attack - SQL Injection': 23, 'Web Attack - SQL Injection - Attempted': 24, 'Web Attack - XSS': 25, 'Web Attack - XSS - Attempted': 26}
Dropping rare classes: [15, 13, 24, 5]
Remaining classes: [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 16, 17, 18, 19, 20, 21, 22, 25, 26]

Applying SMOTE...
After SMOTE: (2373210, 101)

Training XGBoost...


AttributeError: property 'classes_' of 'XGBClassifier' object has no setter

In [6]:
# ============================================================
# CICIDS — OPTIMIZED, FAST, PRODUCTION-READY TRAINING PIPELINE
# ============================================================

import os, math, random, numpy as np, pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================
# CONFIG
# ============================================================

TRAIN_PATH = r"C:\Users\haris\Downloads\split_path\training.csv"
TEST_PATH  = r"C:\Users\haris\Downloads\split_path\testing.csv"

MODEL_DIR = "model_cicids"
OUT_DIR   = "CICIDS_output"
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

LABEL_COL = "Attack"

RANDOM_STATE = 42
FAST_SAMPLE = 150000     # reduce dataset automatically to avoid 700-minute freeze
SMOTE_NEIGH = 3


print("=========== LOADING RAW DATA ===========")
train_df = pd.read_csv(TRAIN_PATH)
test_df  = pd.read_csv(TEST_PATH)

print("Train:", train_df.shape)
print("Test :", test_df.shape)


# ============================================================
# REMOVE NON-FEATURE COLUMNS (IP, timestamp, MAC, flowid)
# ============================================================

drop_keywords = ["flow id", "src ip", "dst ip", "timestamp", "src mac", "dst mac"]

drop_cols = [c for c in train_df.columns if any(k in c.lower() for k in drop_keywords)]

train_df.drop(columns=drop_cols, inplace=True, errors="ignore")
test_df.drop(columns=drop_cols, inplace=True, errors="ignore")

print("After cleaning:", train_df.shape, test_df.shape)


# ============================================================
# AUTO-DOWNSAMPLE FOR SPEED (150k rows)
# ============================================================

def safe_downsample(df, n=150000):
    if len(df) > n:
        return df.sample(n=n, random_state=RANDOM_STATE).reset_index(drop=True)
    return df.reset_index(drop=True)

train_df = safe_downsample(train_df, FAST_SAMPLE)
test_df  = safe_downsample(test_df, FAST_SAMPLE)

print("After downsampling:", train_df.shape, test_df.shape)


# ============================================================
# CLEAN LABELS + REMOVE BROKEN CLASSES
# ============================================================

train_df[LABEL_COL] = train_df[LABEL_COL].astype(str)
test_df[LABEL_COL]  = test_df[LABEL_COL].astype(str)

# Keep common classes
common = sorted(list(set(train_df[LABEL_COL]) & set(test_df[LABEL_COL])))
train_df = train_df[train_df[LABEL_COL].isin(common)]
test_df  = test_df[test_df[LABEL_COL].isin(common)]

# Remove classes with too few samples (SMOTE needs ≥ k+1)
vc = train_df[LABEL_COL].value_counts()
rare = vc[vc < (SMOTE_NEIGH + 1)].index.tolist()

if rare:
    print("Removing rare classes:", rare)
    train_df = train_df[~train_df[LABEL_COL].isin(rare)]
    test_df  = test_df[~test_df[LABEL_COL].isin(rare)]


# ============================================================
# TARGET ENCODER
# ============================================================

all_labels = pd.concat([train_df[LABEL_COL], test_df[LABEL_COL]], axis=0).astype(str)
classes = sorted(all_labels.unique())
target_map = {c: i for i, c in enumerate(classes)}

train_df[LABEL_COL] = train_df[LABEL_COL].map(target_map)
test_df[LABEL_COL]  = test_df[LABEL_COL].map(target_map)

pickle.dump(target_map, open(f"{MODEL_DIR}/target_encoder.pkl", "wb"))
print("\nSaved target encoder.")


# ============================================================
# SAFE CATEGORICAL ENCODING
# ============================================================

def safe_label_encoder(series):
    uniq = sorted(series.astype(str).unique())
    table = {v: i for i, v in enumerate(uniq)}
    def transform(vals):
        return np.array([table.get(str(v), len(table)) for v in vals])
    return table, transform

encoders = {}
cat_cols = [c for c in train_df.columns if train_df[c].dtype == object]

for col in cat_cols:
    table, transform = safe_label_encoder(train_df[col])
    train_df[col] = transform(train_df[col])
    test_df[col]  = transform(test_df[col])
    encoders[col] = table

pickle.dump(encoders, open(f"{MODEL_DIR}/encoders.pkl", "wb"))
print("Saved safe encoders.")


# ============================================================
# SPLIT X / y
# ============================================================

X_train_df = train_df.drop(columns=[LABEL_COL])
y_train = train_df[LABEL_COL]

X_test_df  = test_df.drop(columns=[LABEL_COL])
y_test = test_df[LABEL_COL]

# Align cols
X_train_df, X_test_df = X_train_df.align(X_test_df, join="left", axis=1, fill_value=0)

# Scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_df)
X_test  = scaler.transform(X_test_df)

pickle.dump(scaler, open(f"{MODEL_DIR}/scaler.pkl", "wb"))
print("Saved scaler.")


# ============================================================
# FEATURE LIST (NO CUCKOO — FAST VERSION)
# ============================================================

feature_list = list(X_train_df.columns)
pickle.dump(feature_list, open(f"{MODEL_DIR}/feature_list.pkl", "wb"))
print("Saved feature_list.")


# ============================================================
# BALANCE WITH SAFE SMOTE
# ============================================================

sm = SMOTE(k_neighbors=SMOTE_NEIGH, random_state=RANDOM_STATE)
X_sm, y_sm = sm.fit_resample(X_train, y_train)

print("After SMOTE:", X_sm.shape)


# ============================================================
# TRAIN FAST & STABLE XGBOOST
# ============================================================

xgb = XGBClassifier(
    tree_method="hist",
    n_estimators=120,
    max_depth=12,
    learning_rate=0.08,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=RANDOM_STATE,
    eval_metric="mlogloss",
    n_jobs=-1
)

xgb.fit(X_sm, y_sm)
pickle.dump(xgb, open(f"{MODEL_DIR}/xgb_model.pkl", "wb"))
print("\n✔ Saved XGBoost model.")


# ============================================================
# SAVE LIME BACKGROUND
# ============================================================

bg_idx = np.random.choice(len(X_train), size=min(1200, len(X_train)), replace=False)
lime_bg = X_train[bg_idx]
pickle.dump(lime_bg, open(f"{MODEL_DIR}/lime_background.pkl", "wb"))

print("Saved LIME background.")


# ============================================================
# FINAL EVALUATION
# ============================================================

y_pred = xgb.predict(X_test)

acc  = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted")

print("\n=============== RESULTS ===============")
print("Accuracy :", acc)
print("Precision:", prec)
print("Recall   :", rec)
print("F1 Score :", f1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=classes))

print("\nAll CICIDS model files saved in:", MODEL_DIR)


=========== LOADING RAW DATA ===========
Train: (1469979, 106)
Test : (629992, 106)
After cleaning: (1469979, 102) (629992, 102)
After downsampling: (150000, 102) (150000, 102)
Removing rare classes: ['Infiltration']

Saved target encoder.
Saved safe encoders.
Saved scaler.
Saved feature_list.
After SMOTE: (2264320, 101)

✔ Saved XGBoost model.
Saved LIME background.

=============== RESULTS ===============
Accuracy : 0.9997733257775259
Precision: 0.9997641401639055
Recall   : 0.9997733257775259
F1 Score : 0.9997665200223611

Classification Report:
                                      precision    recall  f1-score   support

                              BENIGN       1.00      1.00      1.00    113013
                              Botnet       1.00      1.00      1.00        44
                  Botnet - Attempted       1.00      1.00      1.00       296
                                DDoS       1.00      1.00      1.00      6856
                       DoS GoldenEye       1.00      1